In [4]:
import pandas as pd

df = pd.read_csv(r'C:\Personal\AIML\LiveProjects\RAG\customer_care_calls_with_notes.csv')
print(df.shape)
df = df[df['ResolutionTimeMinutes'].notna()]  # remove any not available values
data = df.to_dict('records')
print(df.shape)
df

(1000, 11)
(500, 11)


,CallID,CustomerID,CustomerName,CallTimestamp,IssueType,IssueDescription,ResolutionStatus,ResolutionTimeMinutes,AgentID,AgentPerformanceRating,Notes
0,NET32Z8GLG,70BR8GXN,Colton Evans,2024-08-16 14:39:37,Interest Rate Clarification,Customer reported: Interest Rate Clarification.,Closed Without Resolution,30.0,6IO8JJ,4.2,Customer reported an error in Interest Rate Cl...
3,LM8UOURK06,AMC3Y44D,Scott Gregory,2024-05-14 04:42:30,Pre-Closure Request,Customer reported: Pre-Closure Request.,Resolved,107.0,M813YA,1.8,Customer was unhappy with Pre-Closure Request....
5,4JLBII2X23,H9I03MIY,Lisa Norton,2023-12-12 09:36:37,Document Submission Error,Customer reported: Document Submission Error.,Escalated,99.0,BOXUP5,2.6,Customer expressed urgency regarding Document ...
8,GCEB2KDK68,OJAWKI8I,Erica Perez,2024-10-20 19:37:15,Document Submission Error,Customer reported: Document Submission Error.,Closed Without Resolution,115.0,JQAAFW,4.7,Customer provided incomplete details for Docum...
9,KE1N3SJI9R,IJWOPAFB,Zachary Conley,2024-05-20 12:12:40,EMI Calculation Assistance,Customer reported: EMI Calculation Assistance.,Escalated,45.0,1LW859,3.0,Customer was unhappy with EMI Calculation Assi...
...,...,...,...,...,...,...,...,...,...,...,...
992,ZB9KOMDY4C,3IF86V9O,Alice Lucas,2024-01-26 19:45:35,Repayment Schedule Query,Customer reported: Repayment Schedule Query.,Resolved,45.0,0SWBM9,1.6,Customer provided incomplete details for Repay...
993,2QQ9YSQHWG,YR3A8W4F,Mr. Ronald Ortega,2024-02-25 17:23:35,Account Login Issue,Customer reported: Account Login Issue.,Escalated,12.0,2T6EV2,3.4,Customer asked for clarification on Account Lo...
994,LF1PUSW9O1,1V0F1PRK,Zachary Soto,2024-01-15 23:51:20,Overdue Payment Assistance,Customer reported: Overdue Payment Assistance.,Pending,94.0,VC4Z5I,3.0,Customer reported an error in Overdue Payment ...
996,O49TFZN6GK,QI6DDHKQ,Robert Cunningham,2024-07-09 16:18:22,Fraudulent Transaction Report,Customer reported: Fraudulent Transaction Report.,Closed Without Resolution,92.0,22HLDT,3.3,Customer provided incomplete details for Fraud...


In [5]:
pip install qdrant-client sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [ ]:
pip install --upgrade transformers

In [7]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [8]:
# print examples of embedding for understanding purposes along with dimensions w.r.t to the model.

sentence1= "The weather is sunny today"
sentence2="It's a bright and clear day outside"

embedding1=encoder.encode(sentence1)
embedding2=encoder.encode(sentence2)

print(embedding1.shape)
print(embedding1)



(384,)
[-2.04103030e-02  1.08141162e-01  9.42866802e-02  7.14024454e-02
  5.69653846e-02 -5.27100042e-02  9.92613286e-02 -9.13375318e-02
 -5.29344231e-02  1.04881730e-03  3.98172019e-03 -2.87010837e-02
 -5.54135628e-03  4.15268131e-02  8.35011303e-02  6.05910085e-02
 -7.91749451e-03  4.95219138e-04 -3.41409668e-02  3.58934253e-02
 -1.25607088e-01  1.05735539e-02 -6.49580881e-02  1.21010663e-02
  1.84675250e-02  6.07590750e-02  1.52756432e-02 -2.72912486e-03
 -3.94006819e-02 -2.87972428e-02 -1.84790809e-02  4.05811593e-02
 -1.26326671e-02 -5.33437263e-03 -1.62990056e-02 -7.66631812e-02
 -2.19482984e-02 -1.45269752e-01 -9.05513540e-02  8.65175501e-02
 -3.85826593e-03 -9.83408391e-02 -1.69352796e-02 -5.76813472e-03
  1.15468893e-02 -5.06267734e-02 -5.62747493e-02  6.93700165e-02
  1.35780692e-01 -4.42721061e-02 -5.80594465e-02  9.64029785e-03
 -8.24744720e-03 -4.33323160e-03 -6.06535897e-02  1.03219427e-01
  1.56152053e-02 -1.97531283e-02  6.04999959e-02 -2.82947961e-02
  1.26504395e-02  

In [9]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [10]:
# Check if the collection exists, delete it if it does, and then create a new one
if qdrant.collection_exists("customer_calls"):
    qdrant.delete_collection("customer_calls")

# Create the collection with the specified vector configuration
qdrant.create_collection(
    collection_name="customer_calls",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by the model
        distance=models.Distance.COSINE
    )
)


True

In [11]:
import json

# Fetch collection details
collection_info = qdrant.get_collection("customer_calls")

# Convert the collection info to a JSON-compatible dictionary and pretty-print it
formatted_info = json.dumps(collection_info.dict(), indent=4)
print(formatted_info)

{
    "status": "green",
    "optimizer_status": "ok",
    "vectors_count": null,
    "indexed_vectors_count": 0,
    "points_count": 0,
    "segments_count": 1,
    "config": {
        "params": {
            "vectors": {
                "size": 384,
                "distance": "Cosine",
                "hnsw_config": null,
                "quantization_config": null,
                "on_disk": null,
                "datatype": null,
                "multivector_config": null
            },
            "shard_number": null,
            "sharding_method": null,
            "replication_factor": null,
            "write_consistency_factor": null,
            "read_fan_out_factor": null,
            "on_disk_payload": null,
            "sparse_vectors": null
        },
        "hnsw_config": {
            "m": 16,
            "ef_construct": 100,
            "full_scan_threshold": 10000,
            "max_indexing_threads": 0,
            "on_disk": null,
            "payload_m": null
   

In [12]:
# Fetch collection details
collection_info = qdrant.get_collection("customer_calls")

# Print the points count
print("Number of points stored:", collection_info.points_count)


Number of points stored: 0


In [13]:
# Retrieve points from the collection (for example, get 5 points)
points = qdrant.scroll(
    collection_name="customer_calls",
    limit=5
)

# Print the retrieved points
print("Stored data:", points)


Stored data: ([], None)


In [14]:
print("Indexed vectors count:", collection_info.indexed_vectors_count)


Indexed vectors count: 0


In [16]:
# vectorize!

qdrant.upload_records(
    collection_name="customer_calls",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["Notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

C:\Users\Rishi Khanna\AppData\Local\Temp\ipykernel_19416\1158369694.py:3: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [17]:
# Fetch collection details
collection_info = qdrant.get_collection("customer_calls")

# Print the points count
print("Number of points stored:", collection_info.points_count)

Number of points stored: 500


In [19]:
# Inspect the structure of the retrieved points
print("Raw points data:", points)
import json

import json

# Retrieve points from the collection
points, count = qdrant.scroll(
    collection_name="customer_calls",
    limit=5
)

# Convert each Record to a dictionary format manually
points_dict = [
    {
        "id": point.id,
        "payload": point.payload,
        "vector": point.vector,
        "shard_key": point.shard_key,
        "order_value": point.order_value
    } for point in points  # Assuming points here is a list of Record objects
]

# Convert the list of dictionaries to JSON format
points_json = json.dumps(points_dict, indent=4)

# Print the points in JSON format
print("Stored data:", points_json)

# Print the indexed vectors count
print("Indexed vectors count:", collection_info.indexed_vectors_count)


Raw points data: ([], None)
Stored data: [
    {
        "id": 0,
        "payload": {
            "CallID": "NET32Z8GLG",
            "CustomerID": "70BR8GXN",
            "CustomerName": "Colton Evans",
            "CallTimestamp": "2024-08-16 14:39:37",
            "IssueType": "Interest Rate Clarification",
            "IssueDescription": "Customer reported: Interest Rate Clarification.",
            "ResolutionStatus": "Closed Without Resolution",
            "ResolutionTimeMinutes": 30.0,
            "AgentID": "6IO8JJ",
            "AgentPerformanceRating": 4.2,
            "Notes": "Customer reported an error in Interest Rate Clarification. Agent reassured resolution within 24 hours."
        },
        "vector": null,
        "shard_key": null,
        "order_value": null
    },
    {
        "id": 1,
        "payload": {
            "CallID": "LM8UOURK06",
            "CustomerID": "AMC3Y44D",
            "CustomerName": "Scott Gregory",
            "CallTimestamp": "2024-05-

In [18]:

hits = qdrant.search(
    collection_name="customer_calls",
    query_vector=encoder.encode("How often customer are confused what to ask customer care agent").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'CallID': '5IQZ63GCPE', 'CustomerID': 'VJA8MX5Y', 'CustomerName': 'Jason Diaz', 'CallTimestamp': '2024-02-04 19:53:22', 'IssueType': 'Interest Rate Clarification', 'IssueDescription': 'Customer reported: Interest Rate Clarification.', 'ResolutionStatus': 'Resolved', 'ResolutionTimeMinutes': 118.0, 'AgentID': '93KOMK', 'AgentPerformanceRating': 4.3, 'Notes': 'Customer expressed urgency regarding Interest Rate Clarification. Agent prioritized the request.'} score: 0.5236214694903907
{'CallID': 'UWASY90FWE', 'CustomerID': 'SQ3MPLZI', 'CustomerName': 'Jessica Stein', 'CallTimestamp': '2024-06-01 23:29:19', 'IssueType': 'Interest Rate Clarification', 'IssueDescription': 'Customer reported: Interest Rate Clarification.', 'ResolutionStatus': 'Escalated', 'ResolutionTimeMinutes': 77.0, 'AgentID': 'C68TW9', 'AgentPerformanceRating': 2.7, 'Notes': 'Customer expressed urgency regarding Interest Rate Clarification. Agent prioritized the request.'} score: 0.5236214694903907
{'CallID': 'FMNGI6V6EL'